<a href="https://colab.research.google.com/github/dayviddouglas/ciencia_de_dados_licitacoes_prefeitura_do_recife/blob/test/dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install dash
!pip install plotly-express

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 29.4 MB/s eta 0:00:00


In [2]:
import pandas as pd
import requests as req
from io import StringIO
import urllib
import matplotlib.pyplot as plt
import plotly.subplots as sp
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import dash as dsh
from dash import Dash, html,dcc,Input,Output
import plotly.express as px

file_id= "1-0_rPre-v9jjAg1hwdK_mbfYUQqfCvYS"
url =  f"https://drive.google.com/uc?id={file_id}"


try:
    response = req.get(url)
    response.encoding = 'utf-8'
    response.raise_for_status()  # Lança um erro para respostas não-sucedidas
    # Usando StringIO para converter o texto em um arquivo em memória e, então, lendo com o Pandas
    csv_raw = StringIO(response.text)
    base = pd.read_csv(csv_raw, encoding='utf-8')



except req.RequestException as e:
    print(f"Erro ao acessar o arquivo: {e}")




In [3]:
base

,comissao_licitacao,ano_processolicitatorio,num_processolicitatorio,numero_lote,modalidadeprocessolicitatorio,numerolicitacaomodalidade,anolicitacaomodalidade,orgao_licitante,objeto,cnpj_contratado,...,num_endcontratado,comp_endcontratado,bairro_endcontratado,cidade_endcontratado,uf_endcontratado,valor_total_estimado,valor_totalhomologadolicitacao,valor_licitacao_lote,data_aberturaproposta,data_ultimafaseprocesso
0,COMISSÃO PERMANENTE DE LICITAÇÃO DE EDUCAÇÃO -...,2012,29,1,PREGÃO ELETRÔNICO,27,2012,SECRETARIA DE EDUCAÇÃO,MATERIAL GRÁFICO,758606000190.0,...,301.0,NaN,BULTRINS,OLINDA,PE,333750.00,66358.00,15210.00,2012-12-20 10:00:00,2013-01-07
1,COMISSÃO PERMANENTE DE LICITAÇÃO DE EDUCAÇÃO -...,2012,29,2,PREGÃO ELETRÔNICO,27,2012,SECRETARIA DE EDUCAÇÃO,MATERIAL GRÁFICO,7211171000109.0,...,5327.0,NaN,CANDEIAS,JABOATAO DOS GUARARAPES,PE,333750.00,66358.00,51148.00,2012-12-20 10:00:00,2013-01-07
2,COMISSÃO PERMANENTE DE LICITAÇÃO DE EDUCAÇÃO -...,2012,30,1,PREGÃO ELETRÔNICO,28,2012,SECRETARIA DE EDUCAÇÃO,AGENDA ESCOLAR 2013,13898993000102.0,...,0.0,LOJA 005,CIDADE UNIVERSITARIA,MACEIO,AL,530469.54,357895.02,357895.02,2013-01-02 10:00:00,2013-01-09
3,COMISSÃO PERMANENTE DE LICITAÇÃO DE MATERIAIS ...,2012,23,4,PREGÃO ELETRÔNICO,23,2012,FUNDO MUNICIPAL DE SAUDE,"AQUISIÇÃO DE PNEUS, PARA ATENDER AS NECESSIDAD...",10511406000192.0,...,810.0,LOJA 01,PRAZERES,JABOATAO DOS GUARARAPES,PE,16870.35,1518.00,800.00,2012-12-19 11:00:00,2013-03-01
4,COMISSÃO PERMANENTE DE LICITAÇÃO DE MATERIAIS ...,2012,23,5,PREGÃO ELETRÔNICO,23,2012,FUNDO MUNICIPAL DE SAUDE,"AQUISIÇÃO DE PNEUS, PARA ATENDER AS NECESSIDAD...",10511406000192.0,...,810.0,LOJA 01,PRAZERES,JABOATAO DOS GUARARAPES,PE,16870.35,1518.00,718.00,2012-12-19 11:00:00,2013-03-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19103,GC-SEPLAGTD-003,2024,4,7,PREGÃO ELETRÔNICO,4,2024,FUNDO MUNICIPAL DE SAUDE,"REGISTRO DE PREÇOS, COM VALIDADE DE 12 (DOZE) ...",29868059000188.0,...,215.0,ANDAR 1 SL 103,CENTRO,AFOGADOS DA INGAZEIRA,PE,139923.19,110275.52,26000.00,2024-04-18 09:00:00,2024-05-15
19104,GC-SEPLAGTD-003,2024,4,8,PREGÃO ELETRÔNICO,4,2024,FUNDO MUNICIPAL DE SAUDE,"REGISTRO DE PREÇOS, COM VALIDADE DE 12 (DOZE) ...",29868059000188.0,...,215.0,ANDAR 1 SL 103,CENTRO,AFOGADOS DA INGAZEIRA,PE,139923.19,110275.52,28080.00,2024-04-18 09:00:00,2024-05-15
19105,GC-SEPLAGTD-008,2024,9,1,PREGÃO ELETRÔNICO,1,2024,SECRETARIA DE INFRAESTRUTURA,CONTRATAÇÃO PARCELADA DOS SERVIÇOS DE DEMOLIÇÃ...,70215447000163.0,...,537.0,NaN,CURADO,RECIFE,PE,12472497.12,10048265.12,10048265.12,2024-04-18 14:00:00,2024-05-28
19106,GC-SEPLAGTD-008,2024,13,1,PREGÃO ELETRÔNICO,2,2024,SECRETARIA DE INFRAESTRUTURA,"CAÇAMBA ESTACIONÁRIA, COM CAPACIDADE DE NO MÍN...",10811370000162.0,...,550.0,NaN,CANDEIAS,JABOATAO DOS GUARARAPES,PE,2985899.18,2449956.60,2449956.60,2024-05-06 14:00:00,2024-06-03


In [4]:
base2 = base.copy()

def abreviar_coluna_especifica(base2, coluna_especifica):
    # Verifica se a coluna especificada existe no DataFrame
    if coluna_especifica in base2.columns:
        # Aplica a função de abreviação somente na coluna especificada
        base2[coluna_especifica] = base2[coluna_especifica].apply(lambda x: abreviar_apos_primeiro_nome(x) if isinstance(x, str) else x)
    return base2

def abreviar_apos_primeiro_nome(s):
    partes = s.split(maxsplit=1)
    # Mantém o primeiro nome e abrevia o restante das palavras em grupos de 3 caracteres
    if len(partes) > 1:
        # Abrevia cada palavra do restante da string em grupos de 3 caracteres
        abreviacao = ' '.join([palavra[:3] for palavra in partes[1].split()])
        return partes[0] + ' ' + abreviacao
    else:
        return s

# Supondo que base2 seja um DataFrame do pandas, a função agora será aplicada somente na coluna 'orgao_licitante'.
# O código a seguir é apenas para demonstração e não será executado aqui.

base2_abreviado = abreviar_coluna_especifica(base2, 'orgao_licitante')
base2_abreviado = abreviar_coluna_especifica(base2, 'comissao_licitacao')
base2_abreviado = abreviar_coluna_especifica(base2, 'objeto')


In [5]:

process_pe=base[base["uf_endcontratado"]== "PE"]
process_pe["cidade_endcontratado"].value_counts()

# Criando o dataframe Quantidade de processos por cidade

df_process_cidade = pd.DataFrame(process_pe["cidade_endcontratado"].value_counts())
df_process_cidade.reset_index(inplace=True)
df_process_cidade.columns = ['cidade_endcontratado', 'count']






In [6]:
# Transformando a filtragem cidades cujos os produtos ou serviços participaram de licitações em dataframe.
df_process_cidade = pd.DataFrame(process_pe["cidade_endcontratado"].value_counts())
df_process_cidade.reset_index(inplace=True)
df_process_cidade.columns = ['cidade_endcontratado', 'count']
cincos_cidades_com_mais_process=df_process_cidade.iloc[0:5]


In [7]:
# Criando o gráfico dos processos por cidade.
graf_process_cidade = px.bar(cincos_cidades_com_mais_process, x="cidade_endcontratado", y="count", labels={
                     "cidade_endcontratado": "Cidade",
                     "count": "Quantidade"}, barmode="group", title="Quantidade de processos por cidade.")




In [8]:
# Transformando a filtragem sobre as 5 comissões que mais participaram de solicitações em dataframe.
comissoes_licitantes= pd.DataFrame(base["comissao_licitacao"].value_counts())
comissoes_licitantes.reset_index(inplace=True)
comissoes_licitantes.columns = ['comissao_licitacao', 'count']
cinco_mais_comissoes=comissoes_licitantes.iloc[0:5]



In [9]:
# Criando gráfico das 5 comissões que mais participaram de licitações.
graf_comissoes_licitantes=px.bar(cinco_mais_comissoes, x="comissao_licitacao", y="count", labels={
                     "comissao_licitacao": "Comissão",
                     "count": "Quantidade"}, barmode="group", title="Quantidade de processos por comissão.")

In [10]:
freq_orgao_licitantes=pd.DataFrame(base['orgao_licitante'].value_counts())
freq_orgao_licitantes.reset_index(inplace=True)
freq_orgao_licitantes.columns = ['orgao_licitante', 'count']
cinco_mais_orgaos=freq_orgao_licitantes.iloc[0:5]

In [11]:
# Criando gráfico dos 5 orgãos que mais participaram de licitações.
graf_orgao_licitantes=px.bar(cinco_mais_orgaos, x="orgao_licitante", y="count", labels={
                     "orgao_licitante": "Orgão",
                     "count": "Quantidade"}, barmode="group", title="Quantidade de processos por orgão.")

In [12]:
# Transformando a filtragem sobre os 5 objetos que mais participaram de solicitações em dataframe.
freq_objeto=pd.DataFrame(base["objeto"].value_counts().head(5))
freq_objeto.reset_index(inplace=True)
freq_objeto.columns = ['objeto', 'count']




In [13]:
# Criando o Gráfico de objetos que mais participaram de licitações.
graf_objeto=px.bar(freq_objeto, x="objeto", y="count", labels={
                     "objeto": "Objeto",
                     "count": "Quantidade"}, barmode="group", title="Quantidade de licitações por objeto.")

In [14]:
# Transformando a filtragem sobre os 5 valores homologados que mais participaram de solicitações em dataframe.
freq_val_homologados=pd.DataFrame(base["valor_totalhomologadolicitacao"].value_counts().head(5))
freq_val_homologados.reset_index(inplace=True)
freq_val_homologados.columns = ['valor_totalhomologadolicitacao', 'count']



In [15]:
# Criando o Gráfico de valores Homologados que mais participaram de licitações.
graf_valor_homolog = px.bar(freq_val_homologados,x="valor_totalhomologadolicitacao", y="count", labels={
                     "valor_totalhomologadolicitacao": "Valores Homologados",
                     "count": "Quantidade"}, barmode="group", title="Frequência de valores homologados.")


In [16]:
# Transformando a filtragem sobre os 5 valores por lote que mais participaram de licitações em dataframe.
freq_val_lote=pd.DataFrame(base["valor_licitacao_lote"].value_counts().head(5))
freq_val_lote.reset_index(inplace=True)
freq_val_lote.columns = ['valor_licitacao_lote', 'count']



In [17]:
# Criando o Gráfico de licitações por lote.
graf_valor_lote = px.bar(freq_val_lote,x="valor_licitacao_lote", y="count", labels={
                     "valor_licitacao_lote": "Valor por lote",
                     "count": "Quantidade"}, barmode="group", title="Frequência de valores de licitação por lote." )

In [18]:
lista_grafico = [
             {'id_grafico':'gráfico-processos-pe-por-cidade','df':cincos_cidades_com_mais_process, 'x_grafico':'cidade_endcontratado',
             'y_grafico':'count', 'label1':'Cidade', 'label2':'Quantidade','titulo':"Quantidade de processos por cidade."},

            {'id_grafico':'gráfico-comissao-licitacao','df':cinco_mais_comissoes, 'x_grafico':'comissao_licitacao',
             'y_grafico':'count', 'label1':'Comissão', 'label2':'Quantidade','titulo':"Quantidade de processos por comissão."},

            {'id_grafico':'gráfico-orgao-licitante','df':cinco_mais_orgaos,'x_grafico':'orgao_licitante','y_grafico':'count',
             'label1':'Orgão','label2':'Quantidade','titulo':"Quantidade de processos por orgão."},

             {'id_grafico':'gráfico-objeto','df':freq_objeto,'x_grafico':'objeto',
             'y_grafico':'count', 'label1':'Objeto', 'label2':'Quantidade','titulo':"Quantidade de processos por objeto"},
           ]

In [24]:
app= Dash(__name__)

opcoes=[{'label': 'Processos de Licitação em Pernambuco', 'value': 'gráfico-processos-pe-por-cidade'},
        {'label': 'Comissões que participaram de Licitação', 'value': 'gráfico-comissao-licitacao'},
        {'label': 'Orgãos que participaram de licitação', 'value': 'gráfico-orgao-licitante'},
        {'label': 'Objetos que participaram de licitação', 'value': 'gráfico-objeto'},]



opcoes.append({'label': 'Todos os gráficos', 'value': 'todos_os_graficos'})


app.layout=html.Div(children=[
        html.Div([
        html.A([
      html.Button("analise de dados",style={"border":"none","width":"10vw","background":"rgb(99, 110, 250)","color":"white","borderRadius":"20px","padding":"10px"}),
   ],href="#grafico_s",target="_self"),
    html.A([
        html.Button("valores por lote",style={"border":"none","width":"10vw","background":"rgb(99, 110, 250)","color":"white","borderRadius":"20px","padding":"10px"}),
    ],href="#valor_lote",target="_self"),
    html.A([
        html.Button("valores homologados",style={"border":"none","width":"10vw","background":"rgb(99, 110, 250)","color":"white","borderRadius":"20px","padding":"10px"}),
    ],href="#valor_homolog",target="_self"),
    ],style={"display":"flex","alignItens":"center","justifyContent":"center","gap":"30vw"}),

    html.H1(id="texto_principal", children="Processos de Licitação no Estado de Pernambuco"),
    html.H2(children="Quantidade de processos por cidade"),

    dcc.Dropdown(opcoes,'gráfico-comissao-licitacao', id='drop_down'),

       dcc.Graph(id= "grafico_s"),
    dcc.Graph(id="valor_lote",figure=graf_valor_lote),
    dcc.Graph(id="valor_homolog",figure=graf_valor_homolog),


])


@app.callback(
    Output('grafico_s', 'figure'),
    Input('drop_down', 'value')
)
def update_output(value):
    if value == 'todos_os_graficos':
      fig = make_subplots(rows=2, cols=2, subplot_titles=("Quantidade de processos por cidade",
                                                          "Quantidade de processos por comissão", "Quantidade de processos por orgão",
                                                          "Quantidade de processos por objeto"),
     vertical_spacing=0.1,
    horizontal_spacing=0.1,
    row_heights=[0.6, 0.4] )

      fig.add_trace(go.Bar(x=cincos_cidades_com_mais_process["cidade_endcontratado"], y=cincos_cidades_com_mais_process["count"],
                           name="Quantidade de processos por cidade"), row=1, col=1,)

      fig.add_trace(go.Bar(x=cinco_mais_comissoes['comissao_licitacao'],y=cinco_mais_comissoes['count'], name="Quantidade de processos por comissão"),row=1, col=2)

      fig.add_trace(go.Bar(x=cinco_mais_orgaos['orgao_licitante'],y=cinco_mais_orgaos['count'], name="Quantidade de processos por orgão"), row=2, col=1)

      fig.add_trace(go.Bar(x=freq_objeto['objeto'],y=freq_objeto['count'], name="Quantidade de processos por objeto"), row=2, col=2)
      return fig

    else:
      for i in lista_grafico:
        if i['id_grafico'] == value:
          fig = go.Figure(data=[
              go.Bar(x=i['df'][i['x_grafico']], y=i['df'][i['y_grafico']],
                     name="Quantidade de processos por cidade")
          ])
          fig.update_layout(title=i['titulo'], xaxis_title=i['label1'], yaxis_title=i['label2'])
          return fig


In [25]:
if __name__=='__main__':
    app.run(jupyter_mode="external")

Dash app running on:


<IPython.core.display.Javascript object>

In [21]:
base2 = base.copy()

def abreviar_coluna_especifica(base2, coluna_especifica):
    # Verifica se a coluna especificada existe no DataFrame
    if coluna_especifica in base2.columns:
        # Aplica a função de abreviação somente na coluna especificada
        base2[coluna_especifica] = base2[coluna_especifica].apply(lambda x: abreviar_apos_primeiro_nome(x) if isinstance(x, str) else x)
    return base2

def abreviar_apos_primeiro_nome(s):
    partes = s.split(maxsplit=1)
    # Mantém o primeiro nome e abrevia o restante das palavras em grupos de 3 caracteres
    if len(partes) > 1:
        # Abrevia cada palavra do restante da string em grupos de 3 caracteres
        abreviacao = ' '.join([palavra[:3] for palavra in partes[1].split()])
        return partes[0] + ' ' + abreviacao
    else:
        return s

# Supondo que base2 seja um DataFrame do pandas, a função agora será aplicada somente na coluna 'orgao_licitante'.
# O código a seguir é apenas para demonstração e não será executado aqui.

base2_abreviado = abreviar_coluna_especifica(base2, 'orgao_licitante')
base2_abreviado = abreviar_coluna_especifica(base2, 'comissao_licitacao')
base2_abreviado = abreviar_coluna_especifica(base2, 'objeto')

base2_abreviado
# valor_estimado_orgao = base2.groupby('orgao_licitante')['valor_total_estimado'].sum().reset_index()

# # Gerar o gráfico
# fig = px.bar(valor_estimado_orgao, x='orgao_licitante', y='valor_total_estimado', title='Total Estimado por Órgão Licitante (Abreviado)')
# fig.show()

,comissao_licitacao,ano_processolicitatorio,num_processolicitatorio,numero_lote,modalidadeprocessolicitatorio,numerolicitacaomodalidade,anolicitacaomodalidade,orgao_licitante,objeto,cnpj_contratado,...,num_endcontratado,comp_endcontratado,bairro_endcontratado,cidade_endcontratado,uf_endcontratado,valor_total_estimado,valor_totalhomologadolicitacao,valor_licitacao_lote,data_aberturaproposta,data_ultimafaseprocesso
0,COMISSÃO PER DE LIC DE EDU - CPL (GG,2012,29,1,PREGÃO ELETRÔNICO,27,2012,SECRETARIA DE EDU,MATERIAL GRÁ,758606000190.0,...,301.0,NaN,BULTRINS,OLINDA,PE,333750.00,66358.00,15210.00,2012-12-20 10:00:00,2013-01-07
1,COMISSÃO PER DE LIC DE EDU - CPL (GG,2012,29,2,PREGÃO ELETRÔNICO,27,2012,SECRETARIA DE EDU,MATERIAL GRÁ,7211171000109.0,...,5327.0,NaN,CANDEIAS,JABOATAO DOS GUARARAPES,PE,333750.00,66358.00,51148.00,2012-12-20 10:00:00,2013-01-07
2,COMISSÃO PER DE LIC DE EDU - CPL (GG,2012,30,1,PREGÃO ELETRÔNICO,28,2012,SECRETARIA DE EDU,AGENDA ESC 201,13898993000102.0,...,0.0,LOJA 005,CIDADE UNIVERSITARIA,MACEIO,AL,530469.54,357895.02,357895.02,2013-01-02 10:00:00,2013-01-09
3,COMISSÃO PER DE LIC DE MAT - CPL (GG,2012,23,4,PREGÃO ELETRÔNICO,23,2012,FUNDO MUN DE SAU,AQUISIÇÃO DE PNE PAR ATE AS NEC DA GER DE TRA ...,10511406000192.0,...,810.0,LOJA 01,PRAZERES,JABOATAO DOS GUARARAPES,PE,16870.35,1518.00,800.00,2012-12-19 11:00:00,2013-03-01
4,COMISSÃO PER DE LIC DE MAT - CPL (GG,2012,23,5,PREGÃO ELETRÔNICO,23,2012,FUNDO MUN DE SAU,AQUISIÇÃO DE PNE PAR ATE AS NEC DA GER DE TRA ...,10511406000192.0,...,810.0,LOJA 01,PRAZERES,JABOATAO DOS GUARARAPES,PE,16870.35,1518.00,718.00,2012-12-19 11:00:00,2013-03-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19103,GC-SEPLAGTD-003,2024,4,7,PREGÃO ELETRÔNICO,4,2024,FUNDO MUN DE SAU,REGISTRO DE PRE COM VAL DE 12 (DO MES PAR AQU ...,29868059000188.0,...,215.0,ANDAR 1 SL 103,CENTRO,AFOGADOS DA INGAZEIRA,PE,139923.19,110275.52,26000.00,2024-04-18 09:00:00,2024-05-15
19104,GC-SEPLAGTD-003,2024,4,8,PREGÃO ELETRÔNICO,4,2024,FUNDO MUN DE SAU,REGISTRO DE PRE COM VAL DE 12 (DO MES PAR AQU ...,29868059000188.0,...,215.0,ANDAR 1 SL 103,CENTRO,AFOGADOS DA INGAZEIRA,PE,139923.19,110275.52,28080.00,2024-04-18 09:00:00,2024-05-15
19105,GC-SEPLAGTD-008,2024,9,1,PREGÃO ELETRÔNICO,1,2024,SECRETARIA DE INF,CONTRATAÇÃO PAR DOS SER DE DEM PAR DIV EDI NO ...,70215447000163.0,...,537.0,NaN,CURADO,RECIFE,PE,12472497.12,10048265.12,10048265.12,2024-04-18 14:00:00,2024-05-28
19106,GC-SEPLAGTD-008,2024,13,1,PREGÃO ELETRÔNICO,2,2024,SECRETARIA DE INF,CAÇAMBA EST COM CAP DE NO MÍN 5M³ PAR COL TRA ...,10811370000162.0,...,550.0,NaN,CANDEIAS,JABOATAO DOS GUARARAPES,PE,2985899.18,2449956.60,2449956.60,2024-05-06 14:00:00,2024-06-03
